In [57]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tensorflow as tf

from collections import defaultdict
from io import StringIO
from PIL import Image

In [58]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

from utils import label_map_util

from utils import visualization_utils as vis_util

In [59]:
#PATH_TO_CKPT = 'train_goods_4/frozen_inference_graph_61434.pb'
PATH_TO_CKPT = 'train_goods_5/frozen_inference_graph_4719.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('train_goods_4', 'goods_label_map.pbtxt')

IMAGE_DIR = '/home/source/models/research/object_detection/test_goods/test'

NUM_CLASSES = 10

In [60]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [61]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [62]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [63]:
with detection_graph.as_default():
  config = tf.ConfigProto()
  # config.gpu_options.allow_growth = True
  config.gpu_options.per_process_gpu_memory_fraction = 0.1  # 占用GPU50%的显存
  with tf.Session(graph=detection_graph,config=config) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    
    dirlist = os.listdir(IMAGE_DIR) #列出文件夹下所有的目录与文件
    for i in range(0,len(dirlist)):
      image_dir = os.path.join(IMAGE_DIR,dirlist[i])
      if os.path.isdir(image_dir):
        print('output_dir:{}'.format(image_dir))
        imagelist = os.listdir(image_dir)
        for j in range(0,len(imagelist)):
          if imagelist[j].split('_')[0] == 'visual':
            continue
          a, b = os.path.splitext(imagelist[j])
          if b != '.jpg' and b != '.JPG':
            continue
          image_path = os.path.join(image_dir,imagelist[j])
          if os.path.isfile(image_path):
              output_image_path = os.path.join(image_dir,'visual_'+imagelist[j])
              if os.path.isfile(output_image_path):
                continue
              image = Image.open(image_path)
              if image.mode != 'RGB':
                image = image.convert('RGB')
              # the array based representation of the image will be used later in order to prepare the
              # result image with boxes and labels on it.
              image_np = load_image_into_numpy_array(image)
              # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
              image_np_expanded = np.expand_dims(image_np, axis=0)
              # Actual detection.
              (boxes, scores, classes) = sess.run(
                  [detection_boxes, detection_scores, detection_classes],
                  feed_dict={image_tensor: image_np_expanded})
              # Visualization of the results of a detection.
              vis_util.visualize_boxes_and_labels_on_image_array(
                  image_np,
                  np.squeeze(boxes),
                  np.squeeze(classes).astype(np.int32),
                  np.squeeze(scores),
                  category_index,
                  use_normalized_coordinates=True,
                  line_thickness=8)
              output_image = Image.fromarray(image_np)
              output_image.save(output_image_path)
              print('generate output_image:{}'.format(output_image_path))

output_dir:/home/source/models/research/object_detection/test_goods/test/meinianda
generate output_image:/home/source/models/research/object_detection/test_goods/test/meinianda/visual_WIN_20171106_14_49_06_Pro.jpg
generate output_image:/home/source/models/research/object_detection/test_goods/test/meinianda/visual_WIN_20171106_14_49_08_Pro.jpg
generate output_image:/home/source/models/research/object_detection/test_goods/test/meinianda/visual_WIN_20171106_14_49_11_Pro.jpg
generate output_image:/home/source/models/research/object_detection/test_goods/test/meinianda/visual_WIN_20171106_14_49_13_Pro.jpg
generate output_image:/home/source/models/research/object_detection/test_goods/test/meinianda/visual_WIN_20171106_14_49_15_Pro.jpg
generate output_image:/home/source/models/research/object_detection/test_goods/test/meinianda/visual_WIN_20171106_14_49_17_Pro.jpg
generate output_image:/home/source/models/research/object_detection/test_goods/test/meinianda/visual_WIN_20171106_14_49_20_Pro.jpg
